<a href="https://colab.research.google.com/github/zhangxingeng/Capstone/blob/main/Cofilter_Xingeng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import pandas as pd
import numpy as np
import pickle
projectPath = "/content/gdrive/MyDrive/Capstone/co_filter"

In [ ]:
initial_co_filter_df_aquire = False

In [ ]:
if initial_co_filter_df_aquire: # Parse datafram to customerID, ItemID, Rating where rating is just count of purchase
    # article = pd.read_csv("/content/gdrive/MyDrive/Capstone/cleaned_data/articles_clean.csv")
    # customer = pd.read_csv("/content/gdrive/MyDrive/Capstone/cleaned_data/customers_clean.csv")
    trans = pd.read_csv("/content/gdrive/MyDrive/Capstone/cleaned_data/transactions_train.csv")

    customArticlePair = trans[["customer_id", "article_id"]] #.head(1000)
    countPlaceholder = pd.Series([1 for x in range(len(customArticlePair))])
    customArticlePair.loc[:, "rating"] = countPlaceholder # Add a column of purchase count
    # customArticlePair.to_csv("/content/gdrive/MyDrive/Capstone/co_filter/customer_article_pair.csv",index=False)
    countGroup = customArticlePair.groupby(["customer_id", "article_id"]).count() # count purchase numbers
    countGroup.to_csv("/content/gdrive/MyDrive/Capstone/co_filter/customer_article_count.csv") # Save Counts to csv file

In [ ]:
encode_all_ids = False

In [ ]:
def index_continuous_encode(df, colName):
    ''' Encode original ids into continious ids for future pivoting '''
    current = 0
    idEncodeDict = {}
    for i in df.index:
        idOrigin = df.at[i, colName]
        if idOrigin not in idEncodeDict.keys():
            idEncodeDict[idOrigin] = current
            current += 1
        df.loc[i, colName] = idEncodeDict[idOrigin]
    return idEncodeDict

def get_n_user_rating(df, n):
    ''' Sample all movies viewed by n users, better partial prediction than random pick'''
    dfSampledUser = df['customer_id'].sample(n=n)
    selected = df.loc[df['customer_id'].isin(dfSampledUser)].copy()
    return selected

def pickle_dump(path, d):
    with open(path, 'wb+') as f:
        pickle.dump(d, f)

In [ ]:
if encode_all_ids:
    encodes = {}
    userCount = 20
    countDf = pd.read_csv(f"{projectPath}/customer_article_count.csv")
    dfSample = get_n_user_rating(countDf, userCount) # get all purchase of random n users
    # tester = dfSample.sort_values(by='rating', ascending=False).head(100)
    encodes["customer_id"] = index_continuous_encode(dfSample, 'customer_id')
    encodes["article_id"] = index_continuous_encode(dfSample, 'article_id')
    pickle_dump(f"{projectPath}/customer_id_encoding_{userCount}_user.pkl", encodes["customer_id"])
    pickle_dump(f"{projectPath}/article_id_encoding_{userCount}_user.pkl", encodes["article_id"])
    dfSample.to_csv(f"{projectPath}/sampled_{userCount}_user_all_purchase.csv") # Save Counts to csv file

In [ ]:
def pickle_load(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

if False: # Reminder, For future use
    encodes = {}
    encodes["customer_id"] = pickle_load(f"{projectPath}/customer_id_encoding_{getUserCount}_user.pkl")
    encodes["article_id"] = pickle_load(f"{projectPath}/article_id_encoding_{getUserCount}_user.pkl")

In [ ]:
normalize_rating = True

In [ ]:
def normalize_purchase_count(df):
    df_matrix = pd.pivot_table(df, values='rating', index='customer_id', columns='article_id')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    print(d.head())
    # d.index.names = ['scaled_purchase_freq']
    # return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [ ]:
if normalize_rating:
    userCount = 20
    dfSample = pd.read_csv(f"{projectPath}/sampled_{userCount}_user_all_purchase.csv") # Save Counts to csv file
    df_matrix = pd.pivot_table(dfSample, values='rating', index='customer_id', columns='article_id')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    df_matrix_norm = df_matrix_norm.reset_index() # get customer_id out from being row index for unpivoting
    ratingNormDf = pd.melt(df_matrix_norm, id_vars=['customer_id'], value_name='rating_norm').dropna() # unpivot article_ids into tuples
    ratingNormDf.to_csv(f"{projectPath}/normalized_rating_triples.csv") # Save to csv file
    display(ratingNormDf.head())

,customer_id,article_id,rating_norm
1341,1,67,1.0
1354,14,67,0.0
1521,1,76,1.0
1523,3,76,0.0
1538,18,76,0.2


In [ ]:
split_data = True
!pip -q install sklearn
!pip -q install turicreate
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 86.4 MB 72 kB/s 
     |████████████████████████████████| 3.5 MB 30.7 MB/s 
     |████████████████████████████████| 322 kB 41.9 MB/s 
     |████████████████████████████████| 3.6 MB 9.3 MB/s 
     |████████████████████████████████| 18.3 MB 1.4 MB/s 
     |████████████████████████████████| 449 kB 10.3 MB/s 
     |████████████████████████████████| 20.1 MB 1.6 MB/s 
     |████████████████████████████████| 2.9 MB 53.0 MB/s 
     |████████████████████████████████| 3.8 MB 23.0 MB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.16.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.18.5 which is incompatible.
librosa 0.8.1 requires resampy>=0.2.2, but you have resampy 0.2.

In [ ]:
import turicreate as tc
def split_data(data, test_ratio=0.2):
    train, test = train_test_split(data, test_size =test_ratio)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [ ]:
if split_data:
    ratingNormDf = pd.read_csv(f"{projectPath}/normalized_rating_triples.csv")
    train_data_norm, test_data_norm = split_data(ratingNormDf)


def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, user_id=user_id, item_id=item_id, target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, user_id=user_id, item_id=item_id, target=target, similarity_type='cosine')
    elif name == 'pearson':
            model = tc.item_similarity_recommender.create(train_data, user_id=user_id, item_id=item_id, target=target, similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

user_id = 'customer_id'
item_id = 'article_id'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset